### 1) 라이브러리 설치

In [ ]:
#poetry add langchain_community chromadb

### 2) OpenAI 인증키 설정
https://openai.com/

In [ ]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

##### Chroma 간단한 예제
* Chroma DB에 텍스트 파일을 저장하고 검색하는 코드

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader  # 텍스트 파일 로더
from langchain_openai.embeddings import OpenAIEmbeddings  # OpenAI 임베딩 사용
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 텍스트 분할기
from langchain_chroma import Chroma  # 벡터 DB (Chroma) 사용


# 2. 벡터 데이터베이스 저장 경로 설정
DB_PATH = "./db/chroma_db"

# 3. 텍스트 파일을 로드하고 문서를 분할하는 함수 정의
def load_and_split_text(file_path, splitter):
    """
    주어진 텍스트 파일을 로드한 후, 설정된 Splitter를 사용하여 문서를 나누는 함수.
    
    Args:
        file_path (str): 로드할 파일 경로
        splitter (RecursiveCharacterTextSplitter): 텍스트 분할기 객체

    Returns:
        list: 분할된 문서 리스트
    """
    if not os.path.exists(file_path):
        print(f" 파일을 찾을 수 없습니다: {file_path}")
        return []
    
    try:
        loader = TextLoader(file_path)  # 텍스트 파일 로드
        return loader.load_and_split(splitter)  # 분할하여 반환
    except Exception as e:
        print(f" 파일 로드 오류 ({file_path}): {e}")
        return []

# 4. 텍스트 분할기 설정 (600자 단위로 나누고, 100자 겹침 포함)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)

# 5. 두 개의 텍스트 파일 로드 및 분할
split_doc1 = load_and_split_text("data/ai-terminology.txt", text_splitter)
split_doc2 = load_and_split_text("data/finance-terminology.txt", text_splitter)

# 6. 문서 개수 출력
print(f"AI 문서 개수: {len(split_doc1)}")
print(f"금융 문서 개수: {len(split_doc2)}")

# 7. 모든 문서 합치기
all_documents = split_doc1 + split_doc2

# 8. Chroma 벡터 DB 생성 및 저장
try:
    persist_db = Chroma.from_documents(
        documents=all_documents,
        embedding=OpenAIEmbeddings(),  # OpenAI Embeddings 사용
        persist_directory=DB_PATH,  # 벡터 DB 저장 위치 지정
        collection_name="my_vector_db",  # 데이터베이스 컬렉션 이름
    )
    print("Chroma 데이터베이스 저장 완료!")
except Exception as e:
    print(f" Chroma 데이터베이스 저장 오류: {e}")

# 9. 저장된 데이터 확인
try:
    retrieved_docs = persist_db.get()  # Chroma DB에서 데이터 조회
    print(f" 저장된 벡터 개수: {len(retrieved_docs['ids'])}, 타입 {type(retrieved_docs['ids'])}")
except Exception as e:
    print(f" 데이터 조회 오류: {e}")

# 10. 유사도 검색 함수 정의
def search_query(query, k=2):
    """
    사용자 입력(query)에 대해 가장 유사한 문서를 검색하는 함수.

    Args:
        query (str): 검색할 문장 (예: "Transformer 개념 설명")
        k (int, optional): 검색할 문서 개수. Defaults to 2.

    Returns:
        None: 검색 결과를 출력
    """
    try:
        results = persist_db.similarity_search(query, k=k)  # 유사도 검색 수행
        print(f"\n [Query]: {query}\n")
        for i, doc in enumerate(results):
            print(f"🔹 [Result {i+1}]: {doc.page_content[:300]}...\n")  # 검색 결과 출력
    except Exception as e:
        print(f" 검색 오류: {e}")

# 11. 검색 테스트 실행
search_query("Transformer 에 대해 설명해줘", k=2)
search_query("Hedge Fund 에 대해 설명해줘?", k=2)


##### FAISS 간단한 예제
* FAISS DB에 텍스트 파일을 저장하고 검색하는 코드

In [ ]:
import os

from langchain_community.document_loaders import TextLoader  # 텍스트 파일 로더
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 텍스트 분할기
from langchain_community.vectorstores import FAISS  # 벡터 DB (FAISS) 사용
from langchain_ollama import OllamaEmbeddings

# 2. 벡터 데이터베이스 저장 경로 설정
DB_PATH = "../db/faiss_db"

# 3. 텍스트 파일을 로드하고 문서를 분할하는 함수 정의
def load_and_split_text(file_path, splitter):
    """
    주어진 텍스트 파일을 로드한 후, 설정된 Splitter를 사용하여 문서를 나누는 함수.

    Args:
        file_path (str): 로드할 파일 경로
        splitter (RecursiveCharacterTextSplitter): 텍스트 분할기 객체

    Returns:
        list: 분할된 문서 리스트
    """
    if not os.path.exists(file_path):
        print(f" 파일을 찾을 수 없습니다: {file_path}")
        return []
    
    try:
        loader = TextLoader(file_path, encoding="utf-8")  # 텍스트 파일 로드
        return loader.load_and_split(splitter)  # 분할하여 반환
    except Exception as e:
        print(f" 파일 로드 오류 ({file_path}): {e}")
        return []

# 4. 텍스트 분할기 설정 (600자 단위로 나누고, 100자 겹침 포함)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)

# 5. 두 개의 텍스트 파일 로드 및 분할
split_doc1 = load_and_split_text("../data/ai-terminology.txt", text_splitter)
split_doc2 = load_and_split_text("../data/finance-terminology.txt", text_splitter)

# 6. 문서 개수 출력
print(f"AI 문서 개수: {len(split_doc1)}")
print(f"금융 문서 개수: {len(split_doc2)}")


In [ ]:

# 7. 모든 문서 합치기
all_documents = split_doc1 + split_doc2

# 8. FAISS 벡터 DB 생성 및 저장
try:
    ollamaEmbeddings = OllamaEmbeddings(model="bge-m3:latest")

    # FAISS 벡터 DB 생성
    persist_db = FAISS.from_documents(
        documents=all_documents,
        embedding=ollamaEmbeddings,
    )
    
    # 로컬 디스크에 저장
    if not os.path.exists(DB_PATH):
        os.makedirs(DB_PATH)
    persist_db.save_local(DB_PATH)
    
    print("FAISS 데이터베이스 저장 완료!")
except Exception as e:
    print(f" FAISS 데이터베이스 저장 오류: {e}")

# 9. 저장된 데이터 확인
try:
    # FAISS에서 인덱스 정보 확인
    print(f" 저장된 벡터 개수: {persist_db.index.ntotal}")
    print(f" 벡터 차원: {persist_db.index.d}")
    print(f" 인덱스 타입: {type(persist_db.index)}")
except Exception as e:
    print(f" 데이터 조회 오류: {e}")

In [ ]:

# 10. 유사도 검색 함수 정의
def search_query(query, k=5):
    """
    사용자 입력(query)에 대해 가장 유사한 문서를 검색하는 함수.

    Args:
        query (str): 검색할 문장 (예: "Transformer 개념 설명")
        k (int, optional): 검색할 문서 개수. Defaults to 2.

    Returns:
        None: 검색 결과를 출력
    """
    try:
        results = persist_db.similarity_search(query, k=k)  # 유사도 검색 수행
        print(f"\n [Query]: {query}\n")
        for i, doc in enumerate(results):
            print(f"[Result {i+1}]: {doc.page_content[:300]}...\n")  # 검색 결과 출력
    except Exception as e:
        print(f" 검색 오류: {e}")

# 11. 검색 테스트 실행
print('AI 용어 =================')
search_query("Embedding 에 대해 설명해줘", k=3)
print('Finance 용어 =================')
search_query("Hedge Fund 에 대해 설명해줘", k=3)


In [ ]:

# 12. 저장된 FAISS DB 로드 테스트
print("\n" + "="*50)
print("저장된 FAISS DB 로드 테스트")
print("="*50)

try:
    ollamaEmbeddings = OllamaEmbeddings(model="bge-m3:latest")
    # 저장된 FAISS DB 로드
    loaded_db = FAISS.load_local(
        DB_PATH, 
        ollamaEmbeddings,
        allow_dangerous_deserialization=True
    )
    print("FAISS 데이터베이스 로드 완료!")
    
    # 로드된 DB로 검색 테스트
    print(f"로드된 벡터 개수: {loaded_db.index.ntotal}")
    
    # 검색 테스트
    test_results = loaded_db.similarity_search("학습에는 어떤것들이 있나요?", k=3)
    print(f"\n로드된 DB 검색 결과: {test_results[0].page_content[:300]}...")
    
except Exception as e:
    print(f"FAISS DB 로드 오류: {e}")


* [k-최근접이웃 알고리즘](https://ko.wikipedia.org/wiki/K-%EC%B5%9C%EA%B7%BC%EC%A0%91_%EC%9D%B4%EC%9B%83_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98)
* FAISS에서 similarity_search_with_score() 함수의 점수는 실제로는 **거리(distance)**를 나타냅니다.
    * 낮은 점수 = 더 유사함 (거리가 가까움)
    : Result 1 (Score: 0.3795) → 더 유사한 결과
    * 높은 점수 = 덜 유사함 (거리가 멀음)
    : Result 2 (Score: 0.4341) → 덜 유사한 결과

In [ ]:

# 13. 유사도 점수와 함께 검색
print("\n" + "="*50)
print("유사도 점수와 함께 검색")
print("="*50)

def search_with_score(query, k=5):
    """
    유사도 점수와 함께 검색하는 함수
    """
    try:
        results = persist_db.similarity_search_with_score(query, k=k)
        print(f"\n [Query]: {query}\n")
        for i, (doc, score) in enumerate(results):
            print(f"[Result {i+1}] (Score: {score:.4f}):")
            print(f"{doc.page_content[:500]}...\n")
    except Exception as e:
        print(f" 점수 검색 오류: {e}")

# 점수와 함께 검색 테스트
search_with_score("AI 종류는 어떤것들이 있나요?", k=5)


In [ ]:

# 14. FAISS 인덱스 정보 출력
print("\n" + "="*50)
print("FAISS 인덱스 상세 정보")
print("="*50)

try:
    index = persist_db.index
    print(f"인덱스 타입: {type(index).__name__}")
    print(f"총 벡터 수: {index.ntotal}")
    print(f"벡터 차원: {index.d}")
    print(f"훈련 여부: {index.is_trained}")
    print(f"메트릭 타입: {index.metric_type}")
    
    # 메모리 사용량 (추정)
    memory_usage = index.ntotal * index.d * 4 / (1024 * 1024)  # 4 bytes per float, MB 단위
    print(f"추정 메모리 사용량: {memory_usage:.2f} MB")
    
except Exception as e:
    print(f"인덱스 정보 조회 오류: {e}")